In [9]:
!pip install -q langchain langchain-community duckduckgo-search wikipedia yfinance arxiv chromadb faiss-cpu openai tiktoken


In [10]:
import os
from langchain.chat_models import ChatOpenAI
from langchain.agents import create_react_agent, AgentExecutor
from langchain_community.tools import DuckDuckGoSearchResults, WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper
from langchain_community.tools.yahoo_finance_news import YahooFinanceNewsTool
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain import PromptTemplate
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.tools import BaseTool
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from typing import Optional

In [11]:
os.environ["OPENAI_API_KEY"] = ""

In [12]:
!pip install PyPDF

In [14]:
from langchain.document_loaders import PyPDFLoader
import os

pdf_folder = "/content/pdfs"
docs = []

for filename in os.listdir(pdf_folder):
    if filename.endswith(".pdf"):
        pdf_path = os.path.join(pdf_folder, filename)
        try:
            loader = PyPDFLoader(pdf_path)
            docs.extend(loader.load())
        except Exception as e:
            print(f"Skipping file {filename} due to error: {e}")


In [15]:
splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=150)
splits = splitter.split_documents(docs)

In [16]:
embedding = OpenAIEmbeddings(model="text-embedding-3-large")
vectorstore = Chroma.from_documents(splits, embedding, persist_directory="mental_health_db")


/tmp/ipython-input-16-3844981722.py:1: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embedding = OpenAIEmbeddings(model="text-embedding-3-large")


In [17]:
ddg_tool = DuckDuckGoSearchResults()
wiki_tool = WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper())
yfinance_tool = YahooFinanceNewsTool()


In [18]:
from typing import Optional

class MentalHealthRAGTool(BaseTool):
    name: str = "MentalHealthDocumentSearch"
    description: str = (
        "Useful for answering questions about mental health, anxiety, depression based on PDF data."
    )

    def _run(self, query: str) -> str:
        results = vectorstore.similarity_search(query, k=3)
        return "\n\n".join([doc.page_content[:1000] for doc in results])

    async def _arun(self, query: str) -> str:
        return self._run(query)

rag_tool = MentalHealthRAGTool()

In [19]:
from langchain import PromptTemplate

prompt_template = PromptTemplate.from_template(
    """
You are a supportive and evidence-based mental health assistant helping students and the public with anxiety, depression, and emotional well-being.

You can use the following tools to find information:

{tools}

Answer questions using the ReAct reasoning format:

Question: the user's question
Thought: think about what action or info is needed
Action: choose one of [{tool_names}]
Action Input: describe what info you want from the tool
Observation: what the tool returned
... (repeat Thought/Action/Observation up to 3 times)
Final Answer: a short, friendly, and informative answer (~150 words). Be kind and cite sources when needed.

🎯 Keep it concise and conversational. Use everyday language.
💡 Be helpful, never judgmental.
❗ If unsure or the answer is sensitive, suggest speaking to a professional.

⚠️ Note: I am an AI here to inform and support. This is not a medical diagnosis. For urgent or serious concerns, please consult a mental health professional.

Begin!

Question: {input}
{agent_scratchpad}
"""
)


In [20]:
from langchain.chat_models import ChatOpenAI
from langchain.agents import create_react_agent, AgentExecutor

tools = [ddg_tool, wiki_tool, yfinance_tool, rag_tool]

llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0.3, streaming=True)

agent = create_react_agent(llm=llm, tools=tools, prompt=prompt_template)

agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    verbose=True,
    handle_parsing_errors=True,
    return_intermediate_steps=False
)


/tmp/ipython-input-20-1379315320.py:6: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0.3, streaming=True)


In [21]:
print("\nQuery: What are good ways for students to cope with anxiety?")
agent_executor.invoke({"input": "What are good ways for students to cope with anxiety?"})


Query: What are good ways for students to cope with anxiety?


> Entering new AgentExecutor chain...
Thought: Let's provide some evidence-based strategies for coping with anxiety.
Action: MentalHealthDocumentSearch
Action Input: Search for coping strategies for anxiety in studentsMedicine; Health and Medicine Division; Division of Behavioral 
and Social Sciences and Education; Board on Children, Youth, 
and Families; Committee on Applying Lessons of Optimal 
Adolescent Health to Improve Behavioral Outcomes for Youth, 
Kahn, N. F ., & Graham, R. (Eds.). (2019). Promoting Positive 
Adolescent Health Behaviors and Outcomes: Thriving in the 21st 
Century. National Academies Press (US).
120. Centers for Disease Control and Prevention. (2021). 
Coping with Stress. Accessed on November 10, 2021. Retrieved 
from: https://www.cdc.gov/mentalhealth/stress-coping/cope-
with-stress/index.html 
121. The National Institute of Mental Health Information 
Resource Center. (2021). 5 Action Steps for Hel

{'input': 'What are good ways for students to cope with anxiety?',
 'output': 'Agent stopped due to iteration limit or time limit.'}

In [22]:
!pip install fastapi nest-asyncio pyngrok uvicorn

In [23]:
from pyngrok import conf, ngrok

conf.get_default().auth_token = "2zNGwhmownF4gnqhTcZBShdkvQZ_5ugq97chh9YsYmXQUAnUq"


In [25]:
# 📦 Install dependencies (if running in a notebook or fresh env)
# !pip install fastapi nest-asyncio pyngrok uvicorn openai langchain

import nest_asyncio
import csv
import openai
from fastapi import FastAPI
from pydantic import BaseModel
from fastapi.middleware.cors import CORSMiddleware
from pyngrok import ngrok
import uvicorn

from langchain.tools import Tool
from langchain.agents import initialize_agent, create_react_agent, AgentExecutor
from langchain.chat_models import ChatOpenAI

# 🧠 Your tools (must be defined somewhere above this)
# Example: tools = [ddg_tool, wiki_tool, yfinance_tool, rag_tool]
tools = [ddg_tool, wiki_tool, yfinance_tool, rag_tool]

# 🔑 Set OpenAI API Key
openai.api_key = ""  # <-- Replace with your key

# ✅ Setup app
nest_asyncio.apply()
app = FastAPI()

app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"], allow_credentials=True,
    allow_methods=["*"], allow_headers=["*"]
)

# ✅ Input schema
class Query(BaseModel):
    input: str

# ✅ Agent setup
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0.3, streaming=True)
prompt_template = PromptTemplate.from_template(
    """
You are a supportive and evidence-based mental health assistant helping students and the public with anxiety, depression, and emotional well-being.

You can use the following tools to find information:

{tools}

Answer questions using the ReAct reasoning format:

Question: the user's question
Thought: think about what action or info is needed
Action: choose one of [{tool_names}]
Action Input: describe what info you want from the tool
Observation: what the tool returned
... (repeat Thought/Action/Observation up to 3 times)
Final Answer: a short, friendly, and informative answer (~150 words). Be kind and cite sources when needed.

🎯 Keep it concise and conversational. Use everyday language.
💡 Be helpful, never judgmental.
❗ If unsure or the answer is sensitive, suggest speaking to a professional.

⚠️ Note: I am an AI here to inform and support. This is not a medical diagnosis. For urgent or serious concerns, please consult a mental health professional.

Begin!

Question: {input}
{agent_scratchpad}
"""
)

agent = create_react_agent(llm=llm, tools=tools, prompt=prompt_template)
agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    verbose=True,
    handle_parsing_errors=True,
    return_intermediate_steps=True,  # <-- enable this
    max_iterations=5  # or whatever your safe limit is
)

from openai import OpenAI

client = OpenAI()

def evaluate_with_openai(question, answer):
    prompt = f"""
You are an evaluator for a mental health chatbot.

Evaluate the assistant's response based on:
1. Relevance (1–5)
2. Factual Accuracy (1–5)
3. Empathy (1–5)
4. A short comment

Input Question: {question}
Assistant Response: {answer}

Respond ONLY in JSON format like:
{{
  "relevance": 5,
  "factuality": 5,
  "empathy": 4,
  "comment": "Very empathetic and helpful response."
}}
"""

    try:
        response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[
                {"role": "system", "content": "You are a strict evaluator of chatbot responses."},
                {"role": "user", "content": prompt}
            ],
            temperature=0.3
        )
        return eval(response.choices[0].message.content)
    except Exception as e:
        print("Evaluation error:", e)
        return {
            "relevance": 3,
            "factuality": 3,
            "empathy": 3,
            "comment": "Evaluation failed"
        }

# ✅ CSV logger
def log_evaluation(question, answer, scores):
    with open("chat_evaluation_log.csv", mode="a", newline="") as f:
        writer = csv.writer(f)
        writer.writerow([
            question,
            answer,
            scores["relevance"],
            scores["factuality"],
            scores["empathy"],
            scores["comment"]
        ])

# ✅ Main Chat endpoint
@app.post("/chat")
async def chat(query: Query):
    try:
      result = agent_executor.invoke({"input": query.input})
      response = result["output"]
    except Exception as e:
      print("Agent error:", e)
      # Fallback: try to get something useful
      if isinstance(e, OutputParserException) and "intermediate_steps" in e.__dict__:
          last_step = e.intermediate_steps[-1] if e.intermediate_steps else None
          response = f"Agent stopped early. Last step: {last_step}"
      else:
          response = "Sorry, something went wrong during processing."


    # Evaluate & log
    scores = evaluate_with_openai(query.input, response)
    log_evaluation(query.input, response, scores)

    return {
        "response": response,
        "evaluation": scores
    }

# ✅ Expose via ngrok
public_url = ngrok.connect(8000)
print(f"🔗 Public URL: {public_url}")

uvicorn.run(app, port=8000)


🔗 Public URL: NgrokTunnel: "https://6321-34-45-122-194.ngrok-free.app" -> "http://localhost:8000"


INFO:     Started server process [366]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


INFO:     2405:201:a419:bd:6c9a:204a:c010:d645:0 - "OPTIONS /chat HTTP/1.1" 200 OK


> Entering new AgentExecutor chain...
Thought: I need to find information about the 5 C's of mental health.
Action: MentalHealthDocumentSearch
Action Input: Search for the 5 C's of mental healthMental Health and Wellbeing — A Perspective | 6

4
For all individuals, mental, physical
and social health are vital and inter-
woven strands of life. As our under-
standing of this relationship grows, 
it becomes ever more apparent that
mental health is crucial to the overall
well-being of individuals, societies and
countries. Indeed, mental health can
be deﬁned as a state of well-being
enabling individuals to realize their
abilities, cope with the normal stresses
of life, work productively and fruitful-
ly, and make a contribution to their
communities. Unfortunately, in most
parts of the world, mental health and
mental disorders are not accorded
anywhere near the same degree of
importance as physical health. R

INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [366]
